# Calibration of the NCCS pipeline

This document goes through some of the background 

# Introduction

//What needs calibrating?

//What are our observations?

//What does our approach need to do?

//What solution did we pick?

# Methodology

## An example

Let's choose a concrete example that we can follow through this workbook. We'll start with something fairly simple: a two-dimensional parameter space describing asset losses for tropical cyclones.

//Describe example: calibrating TC impact function
//Show the parameter space for an impact function

## What does a Bayesian approach mean?

//Bayesian versus frequentist

//Advantages of a Bayesian approach

## What is an optimisation?

An algorithmic optimisation lets us search a parameter space for the combination of values that best 'solves' some problem that we're interested in. This could be as simple as a linear regression, or as complex as tuning the hyperparemeters of a neural network. In our case we're interested in tuning impact functions in CLIMADA, adjusting their size and shape until the modelled losses best resemble the observations that we have.

Every optimisation problem requires three elements (or some variation on these three elements):

- **The parameter space:** information about all the different variables whose possible values we want to explore, plus any constraints we want to place on their permitted values, or rules about which combinations of values can be used. In this case, these are the parameters that together define our impact function(s).
- **The objective function:** a function (often a statistical model) that takes these parameters as input and maps them to some output. For us, this is a wrapper around part or all of our modelling chain that maps the input parameters to various modelled impacts. In the example this uses the input parameters to create an impact function, and runs CLIMADA with this impact function, outputting losses. The output doesn't have to be a single value, it can be anything you like: for example, average annual loss from an event set, or a vector of losses from every event on every sector, or losses from a single location, or a combination of all these things. 
- **The cost function:** This takes two inputs, the output from the objective function, and someobservational or training data which is considered to be the target value(s) for the objective function. The cost function tells the optimisation how well or poorly each combination of parameters performs compared to the observations. In our example, our observations are event losses from EM-DAT and the cost function calculates the difference between modelled and observed losses and returns a summary statistic (we discuss the choice of this statistic later).

The optimisation then explores the parameter space, attempting to find a combination of parameters that minimises the cost function (and in our case, in as few steps as possible). Most of the content below ends up being about this process to find the combination of parameters that gives the minimum value of the cost function: how to model an approximation of the loss function and how to sample from the parameter space most efficiently.

An optimisation algorithm searches for this the optimal parameter combination that minimises the cost function with an iterative process. For each iteration:

- The algorithm looks at the points it has already sampled from the parameter space and what it knows about the behaviour of the cost function
- It uses some modelling process (see below) to choose another point in the parameter space to sample that will add the most useful information about the location of the function's minimum (discussed below too)
- It runs the model (the objective function) for the new sampled parameters from the parameter space and calculates the value of the cost function
- If some stopping criterion is met (e.g. only small improvements in our knowledge of the cost function, a maximum number of iterations reached) it stops. Otherwise it starts the next iteration.

How the algorithm thinks about the form of the cost function and how it chooses its next sample from the parameter space are big and interesting problems with a lot of solutions. The solution we will choosing is Bayesian optimisation with Gaussian process priors. The Bayesian optimisation is our approach for choosing the next point, the Gaussian process priors refer to how we approximate our the cost function with what we know.

## Why do we use Gaussian processes in optimisation?

A Gaussian process is a good way to approximate any unknown function defined over a continuous domain where we know its value at a finite set of points (e.g. the value of the cost function). In particular, the way a Gaussian process approximates a function is mathematically nice: as awe get more information the approximation gets more and more accurate, it doesn't expect a particular statistical distribution or functional form (which we wouldn't expect in a complex problem like this) and it has, predictable, cheap-to-calculate properties, including marginals and conditionals which are important in Bayesian inference. These is important when we use the approximated function for operations such as choosing the next parameter combination to test.

In straightforward approaches to an optimisation with a Gaussian process, the optimising algorithm will gradually refine its understanding of the parameter space by sampling points from within it. The optimiser will typically look at the local gradient of the function and its Hessian (i.e. second derivatives) to choose the next location to sample. This works well, but there are more efficient ways to search if you really want to reduce the number of times you have to call your objective function (the model)

At the cost of more expensive calculations of where to sample next, a Bayesian process can make better inferences about the combination of parameters that, when sampled, are most likely to improve its knowledge about the cost function's minimum.

## What is Bayesian optimisation?

Bayesian optimisation refers to a particular family of optimisation algorithms that considers the objective function itself as probabilistic, that is, each step of the optimisation isn't considering a single 'best guess', but instead looks at the whole space of possible approximating functions (in our case, the space of Gaussian processes defined on samples from our parameter space). Each function in the space has an associated plausibility quantifying how well it does (or doesn't) agree with the data we have.

A Bayesian optimisation algorithm consists of an iterative process that repeatedly (and very very strategically) samples the parameter space based on what it already knows about the cost function's behaviour (the prior distribution), runs our model for the sampled parameters and calculates the cost function, and then updates its statistical description of the plausibility of all approximating functions with the new information. This updated description is the 'posterior' distribution. Each step of the optimisation algorithm is mathematically a Bayesian update, the core process of all Bayesian inference.

## Why Bayesian optimisation is pretty useful

We choose Bayesian optimisation because it prioritises reaching conclusions in _as few steps as possible_ of the optimisation iteration. Given how expensive it is to run our modelling chain (or even a component of the chain) we want to minimise the number of times we have to run the model. This isn't always a requirement of optimisation routines: many (even most) statistical models are fairly simple and create objective functions that can be evaluated in microseconds. In these cases the model's computational cost isn't a huge factor in algorithm design. Thankfully, a lot of people care about exactly this problem at the moment because a lot of people are exploring hyperparemeter spaces used to design machine learning models, which are usually quite expensive to train.

Bayesian optimisation does this by allowing our algorithm to consider more information (encoded as uncertainty) in its decision-making than other, simpler approaches. Each iteration of the optimisation algorithm is a bigger improvement on the previous step when compared with the stepwise improvements that other algorithms make (usually! It of course depends on you setting up your model and optimisation correctly). Note: a Bayesian approach isn't the only way to accomplish this! But it ends up being quite a neat one, in my opinion at least.

This comes at the cost of more computationally complex overhead to decide on the next sample, but when our model is expensive to run this is definitely worth it.

## The posterior distribution

Describing the cost function as a probabilistic family of functions has other benefits. It means that the algorithm's final output telling us what it has inferred about the cost function's form and the location of its minimum isn't just a single point estimate from the parameter space, we get a full posterior distribution of cost function approximations that we can sample to explore the full range of plausible impact functions generated through the calibration. 

## What is an acquisition function?

//It's another cost function. This one is used to choose the next combination of parameters to sample from the parameter space.

## The algorithm

In this Bayesian formulation of the problem, each step of the iterative search proceeds as in the above outline:

- We want to approximate the cost function with a Gaussian process. The Bayesian approach assumes that there is a non-parametric statistical distribution of cost functions, each with a prior plausibility. If this is the first iteration, the priors are set by the user. Otherwise the posterior of the previous iteration becomes the prior for the next iteration.
- An aquisition function (see above) is used to choose a new point to sample from the parameter space, trying to maximise the additional information we gain from evaluating the objective function at this point.
- The model is run and the cost function calculated
- Our probabilistic model of plausible approximations of the objective function is updated with this new data point to generate a posterior distribution of plausible approximations of the objective function. This approximation will be more accurate and constrained than the prior.
- If some user-provided stopping criterion is met it stops. Otherwise it starts the next iteration.

The optimisation module underlying all our code uses this approach and is described in the documentation for the `bayesian-optimisation` package: https://github.com/bayesian-optimization/BayesianOptimization.

A wrapper around the algorithm has been produced by Lukas Riedel and is in CLIMADA's `calibration` package. Many many thanks to Lukas!

For more information on Bayesian optimisation in general see https://proceedings.neurips.cc/paper_files/paper/2012/file/05311655a15b75fab86956663e1819cd-Paper.pdf


## Summary: why Bayesian?

- It lets us use all available info without making further assumptions
- It works with probability distributions of how plausible different descriptions of our the objective function are, given the samples it has made.
- ... this distribution is _nonparametric_, i.e. it can have an arbitrary and complex functional form, which we would require for a complex multidimensional parameter space like the one we're exploring
- ... and this posterior probability distribution can be sampled as a way of exploring model uncertainty



## How we calibrate a component

//Model description

## How we calibrate the whole pipeline

//description

# A note on uncertainty simulations

One of the lovely things about (a set of) Bayesian posteriors (and also priors) is that they are 'generative'. That means that you can sample from it, as with parametric distributions. This allows us to think of parameter we've calibrated as uncertain, not just a point estimate.

Until now we've just been running model chains using the 'best' point estimates for each parameter. But we can also run simulations with other choices of each parameter. In theory, we could run hundreds of supply chain simulations, each one sampling from the uncertainty distribution of the calibrated parameters. This will give a 'full' range of uncertainty in the modelled results, where the highest impact in the output simulations isn't just from the most intense event in the event set, it's from the most intense event _in the simulation that sampled the most extreme, unlikely impact function_.

CLIMADA's `unsequa` module provides a suite of tools to set up these simulations.

## Storylines

In practice, however, we may decide that these are far too expensive computationally. Currently a full modelling chain takes many many hours to run. While we have a few ideas on how to speed it up (see below), and there are some neat ways to explore parts of the parameter uncertainty space without recalculating all of the impacts, we may decide that we don't have the time or computational resources for a fully probabilistic exploration of the uncertainty space.

If this happens, we would fall back on the well-loved Storylines approach
//explain storylines


## Simulations to explore uncertainty
To save computation time, we probably want to look at the uncertainty around individual componenents of the modelling chain. That is, look at one hazard at a time, and maybe look at uncertainty in direct asset loss and business interruption separately. While this will miss a lot of interaction effects, it's very good for telling a clear story, and it's also good for easier-to-analyse results!

//more?

## How to speed up simulations
Most readers should skip this section. It's a set of notes, pretty technical, and not required for a full understanding of the modelling!

- Deployment on the CelsiusPro VM. This has a _lot_ of computational resources.
- Component-wise uncertainty assessment, as described in the above section.
- Linear scaling of direct impacts is cheap //explain
- Collapsing the event set: //explain 
    - removing years with similar impacts //explain
    - removing years with lower impacts //explain
- Reducing IO costs (mostly the need to read in Exposures multiple times and to store an Impact object's `imp_mat` attribute. Both of these could be avoided after Samuel Juhel's refactor of the Supply Chain module, we hope).



# Version 1: calibrate direct asset loss impact functions

Here we take a very pragmatic approach. The goal is to calibrate everything component-wise, and to calibrate only the components where there is enough data for a meaningful level of certainty in the outputs. In practice, that means calibrating direct asset loss impact functions for each hazard to EM-DAT loss data.

//talk about the chosen cost function

## Missing from the calibration
- Full pipeline calibrations
    - including: calibrating/constraining combined components hazard-BI and hazard-BI-supplychain
    - including: 
- No inclusion of expert judgement as priors (and therefore no treatment of uncertainty in non-calibrated components)
- No vulnerability-based calibration. Tropical cyclones and river flood are calibrated to regions based roughly on continent, following previous calibrations in CLIMADA (see below).


## V1 calibration components
- Tropical cyclone: //info
- River flood: //info
- European windstorm: //info
- Relative crop yield: //info
- Wildfire: not done


# Version 2: add regional vulnerability to direct asset losses

Version 2 is very similar to Version 1, but adds an extra degree of freedom to each calibrated impact function in the form of a vulnerability parameter. This is (proposed) to be taken from the ND-GAIN data that is already used by EBP in their metrics, since it's trusted by the group and ensures a consisten idea of 'vulnerability' between components of the project. It will assign a vulnerability index to each country. (Exactly what we use will take a little experimentationL should it be the raw ND-GAIN index, the country rank, quantile information, or something else? We can run a few quick tests.)

There are two challenges from  adding an extra degree of freedom:\
- **Computational complexity:** //explain, including how the implemented code is chosen iterating through a space where calculations take a long time
- **Parameter uncertainty:** for each additional parameter you add to a statistical model, there is a tradeoff between the explanatory power it provides versus the increasing uncertainty the model parameters, especially in models trained on small datasets, such as we're doing here. The hope is that vulnerability adds a lot of explanatory power. The hope is even that it _reduces_ uncertainty. (We'll see. It's possible for tropical cyclones and river flood since the calibration of these perils are split into regions anyway, which is equivalent to  another degree of freedom). We'll be able to compare the V1 and V2 calibrations and decide where the vulnerability information improves the model.

## V2 calibration components
// These are the same as the V1 components

# Version 3: add expert judgement for uncalibrated components

// Version 3 continues the approach 

# Version 3

Version 3 is the big conceptual shift. Instead of calibrating individual components

// explain

The big change Version 2 looks at combining components when they can meaningfully in